# Group Knockoffs

This tutorial generates group (model-X) knockoffs, which is useful when predictors are highly correlated. The methodology is described in the following paper

> Dai R, Barber R. The knockoff filter for FDR control in group-sparse and multitask regression. InInternational conference on machine learning 2016 Jun 11 (pp. 1851-1859). PMLR.


!!! note

    In the original paper, Dai and Barber only describes how to construct a suboptimal equi-correlated group knockoffs. Here we implement fully generalized alternatives.
    
Currently available options for group knockoffs:
+ `:maxent`: Fully general maximum entropy (maxent) group knockoff, based on coordinate descent.
+ `:mvr`: Fully general minimum variance-based reconstructability (MVR) group knockoff, based on coordinate descent.
+ `:sdp`: This generalizes the equi-correlated group knockoff idea by having $S_j = \gamma_j \Sigma_{(G_j, G_j)}$. Instead of optimizing over all variables in $S$, we optimize over a vector $\gamma_1,...,\gamma_G$. 
+ `:equi`: This implements the equi-correlated idea proposed in [Barber and Dai](https://proceedings.mlr.press/v48/daia16.html), which lets $S_j = \gamma \Sigma_{(G_j, G_j)}$ where $\Sigma_{(G_j, G_j)}$ is the block of $\Sigma$ containing variables in the $j$th group. Thus, instead of optimizing over all variables in $S$, we optimize a scalar $\gamma$. Conveniently, there a simple closed form solution for $\gamma$. For `mvr` and `maxent` group knockoffs, we initialize $S$ using this construction. 


In [14]:
# load packages for this tutorial
using Revise
using Knockoffs
using LinearAlgebra
using Random
using StatsKit
using ToeplitzMatrices
using Distributions

# Data simulation

## Gaussian model-X knockoffs with known mean and covariance

To illustrate, lets simulate data $\mathbf{X}$ with covariance $\Sigma$ and mean $\mu$. Our model is
```math
\begin{aligned}
    X_{p \times 1} \sim N(\mathbf{0}_p, \Sigma)
\end{aligned}
```
where
```math
\begin{aligned}
\Sigma = 
\begin{pmatrix}
    1 & \rho & \rho^2 & ... & \rho^p\\
    \rho & 1 & & ... & \rho^{p-1}\\
    \vdots & & & 1 & \vdots \\
    \rho^p & \cdots & & & 1
\end{pmatrix}
\end{aligned}
```
Given $n$ iid samples from the above distribution, we will generate knockoffs according to 
```math
\begin{aligned}
(X, \tilde{X}) \sim N
\left(0, \ 
\begin{pmatrix}
    \Sigma & \Sigma - diag(s)\\
    \Sigma - diag(s) & \Sigma
\end{pmatrix}
\right)
\end{aligned}
```

Because variables are highly correlated with its neighbors ($\rho = 0.9$), it becomes difficult to distinguish which among a bunch of highly correlated variables are truly causal. Thus, group knockoffs test whether a *group* of variables have any signal should have better power than standard (single-variable) knockoffs. 

First, lets simulate some data

In [3]:
# simulate data
Random.seed!(2023)
n = 250 # sample size
p = 500 # number of features
k = 10  # number of causal variables
Σ = Matrix(SymmetricToeplitz(0.9.^(0:(p-1))))
# Σ = simulate_AR1(p, a=3, b=1)
# Σ = simulate_block_covariance(groups, 0.75, 0.25)
μ = zeros(p)
L = cholesky(Σ).L
X = randn(n, p) * L # design matrix
zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

# Define group memberships

To generate group knockoffs, we need to vector specifying group membership. One can define this vector manually, or use the built-in functions [`hc_partition_groups`](https://biona001.github.io/Knockoffs.jl/dev/man/api/#Knockoffs.hc_partition_groups) or [`id_partition_groups`](https://biona001.github.io/Knockoffs.jl/dev/man/api/#Knockoffs.id_partition_groups). 

In [5]:
groups = hc_partition_groups(X, cutoff = 0.5)

500-element Vector{Int64}:
  1
  1
  1
  2
  2
  2
  2
  3
  3
  3
  3
  3
  4
  ⋮
 93
 93
 93
 93
 94
 94
 94
 95
 95
 96
 96
 96

## Generating group knockoffs

Generate group knockoffs with the exported function [`modelX_gaussian_group_knockoffs`](https://biona001.github.io/Knockoffs.jl/dev/man/api/#Knockoffs.modelX_gaussian_group_knockoffs). Similar to non-group knockoffs, group knockoff accepts keyword arguments `m`, `tol`, `method`, and `verbose` which controls the algorithm's behavior. 

In [12]:
@time Gme = modelX_gaussian_group_knockoffs(
    X, :maxent, groups, μ, Σ, 
    m = 5,              # number of knockoffs per variable to generate
    tol = 0.0001,       # convergence tolerance
    inner_ccd_iter = 1, # optimize every entry of S exactly 1 time before moving on to PCA updates
    inner_pca_iter = 1, # optimize S with respect to pre-computed eigenvectors 1 time before going to CCA updates
    verbose=true);      # whether to print informative intermediate results

Maxent initial obj = -10748.931182611366
Iter 1 (PCA): obj = -8237.523985375447, δ = 0.16612980949819264, t1 = 0.03, t2 = 0.07
Iter 2 (CCD): obj = -7700.4180043840925, δ = 0.03731459026023631, t1 = 0.1, t2 = 0.23, t3 = 0.0
Iter 3 (PCA): obj = -7425.308463255121, δ = 0.060823604038161845, t1 = 0.14, t2 = 0.3
Iter 4 (CCD): obj = -7308.074499758944, δ = 0.019936923421413913, t1 = 0.18, t2 = 0.47, t3 = 0.0
Iter 5 (PCA): obj = -7229.089072737786, δ = 0.036631251375332345, t1 = 0.21, t2 = 0.53
Iter 6 (CCD): obj = -7182.085494609535, δ = 0.009753360860322807, t1 = 0.24, t2 = 0.7, t3 = 0.0
Iter 7 (PCA): obj = -7142.003455889968, δ = 0.02924379554922749, t1 = 0.27, t2 = 0.75
Iter 8 (CCD): obj = -7116.141211182261, δ = 0.008709415093997086, t1 = 0.3, t2 = 0.92, t3 = 0.01
Iter 9 (PCA): obj = -7088.633103966935, δ = 0.021931911964248568, t1 = 0.33, t2 = 0.98
Iter 10 (CCD): obj = -7071.026287747464, δ = 0.007971315641216871, t1 = 0.36, t2 = 1.15, t3 = 0.01
Iter 11 (PCA): obj = -7049.550647201161, δ

+ Here CCD corresponds to optimization each entry ``S_{ij}`` independently, while PCA is a faster update that updates ``S_{new} = S + \delta vv'``. 
+ Users can modify the default behavior by supplying the arguments `inner_pca_iter` and `inner_ccd_iter`. For instance, we can turn off `inner_ccd_iter` to achieve much faster convergence at the sacrifice small accuracy. 
+ ``t_1, t_2, t_3`` are timers, which reveals that the computational bottleneck is in (2), which we dispatch to efficient LAPACK libraries, so the overall performance of our algorithm cannot really be improved. 
    1. ``t_1``: updating cholesky factors
    2. ``t_2``: solving forward-backward equations
    3. ``t_3``: solving off-diagonal 1D optimization problems using Brent's method

The output is a struct with the following fields
```julia
struct GaussianGroupKnockoff{T<:AbstractFloat, BD<:AbstractMatrix, S<:Symmetric} <: Knockoff
    X::Matrix{T} # n × p design matrix
    X̃::Matrix{T} # n × mp matrix storing knockoffs of X
    groups::Vector{Int} # p × 1 vector of group membership
    S::BD # p × p block-diagonal matrix of the same size as Σ. S and (m+1)/m*Σ - S are both psd
    γs::Vector{T} # for suboptimal group construction only. These are scalars chosen so that S_i = γ_i * Σ_i
    m::Int # number of knockoffs per feature generated
    Σ::S # p × p symmetric covariance matrix. 
    method::Symbol # method for solving s
    obj::T # final objective value of group knockoff
end
```
Given this result, lets do a sanity check: is $(m+1)/m\Sigma - S$ positive semi-definite?

In [7]:
eigmin((m+1)/m*Gme.Σ - Gme.S)

0.05110858707177174

## Second order group knockoffs

In practice, we often do not have the true covariance matrix $\Sigma$ and the true means $\mu$. In that case, we can generate second order group knockoffs via the 3 argument function

In [14]:
Gme_second_order = modelX_gaussian_group_knockoffs(X, :maxent, groups);

This will estimate the covariance matrix via a shrinkage estimator, see documentation API for more details. 

## Representative group knockoffs

One can choose a few representatives from each group and generate *representative* group knockoffs, with the following advantage:

+ Dramatically improved computational efficiency, since the group-knockoff optimization problem only needs to be carried out on the representative variables.
+ Improved power over standard group knockoffs, since the exchangeability have to be satisfied for less variables, so the resulting knockoffs are more "flexible"

This model assume that conditional on the group representatives, remaining variables are independent by groups. Although this assumption is not always met, we find that group-FDR is never really violated in our experiments with real or simulated data. 

In [7]:
@time rME = modelX_gaussian_rep_group_knockoffs(
    X, :maxent, μ, Σ, groups,
    m = 5,          # number of knockoffs per variable to generate
    tol = 0.0001,   # convergence tolerance
    verbose=true);  # whether to print informative intermediate results

96 representatives for 500 variables, 96 optimization variables
Iter 1: δ = 0.14953283217899976
Iter 2: δ = 0.1991610624266248
Iter 3: δ = 0.01932169475512019
Iter 4: δ = 0.005068052646704513
Iter 5: δ = 0.0009112831450636683
Iter 6: δ = 0.0001230274318336222
Iter 7: δ = 2.4675893956049855e-5
  0.250789 seconds (15.34 k allocations: 214.701 MiB, 14.04% gc time)


Note that the resulting knockoffs are still $n \times mp$

In [8]:
rME.X̃

250×2500 Matrix{Float64}:
 -0.747196   -1.2342    -0.702566    …   1.66098    1.79071     1.40427
  0.600782    0.109681  -1.2857         -0.81733   -0.876087   -0.539925
 -1.4536     -1.53914   -1.76241        -1.07475   -0.982822   -0.364055
 -1.24691    -0.878209  -0.122253        0.882058   0.698461    1.27731
  0.669513    0.478596   0.718306       -1.11829   -0.958759    0.00439087
 -1.04199    -0.784127  -1.82756     …   0.905066   0.748423    0.339189
 -0.754254   -0.33635    0.442443        0.24998   -0.0987811   0.0899613
 -2.35308    -1.81752   -2.28223        -0.164806  -0.104967   -0.447325
 -2.20415    -2.76933   -2.59485        -0.905337  -0.745101    0.237391
  2.20236     2.1198     1.64855        -1.02309   -1.22663    -0.745322
 -1.46614    -0.198733  -0.508032    …  -1.75448   -2.04408    -1.30121
  0.0185783  -0.123839  -0.524711       -0.169963  -0.0599242   0.0216337
  0.222628    0.110846  -0.0438031      -1.09253   -1.05077    -1.2574
  ⋮                       

## Lasso Example

Lets see the empirical power and FDR group knockoffs over 10 simulations when
+ the targer FDR is 10%
+ we generate $m=5$ knockoffs per feature
+ ``\beta_j \sim \pm 0.25`` for 10 causal ``j``s

Note power and FDR is defined at the group level

In [25]:
group_powers, group_fdrs, group_times, group_s = Float64[], Float64[], Float64[], Float64[]

Random.seed!(2022)
for sim in 1:10
    # simulate X
    Random.seed!(sim)
    n = 1000 # sample size
    p = 200  # number of covariates
    k = 10   # number of true predictors
    Σ = Matrix(SymmetricToeplitz(0.9.^(0:(p-1)))) # true covariance matrix
    μ = zeros(p)
    L = cholesky(Σ).L
    X = randn(n, p) * L
    zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

    # define groups
    groups = hc_partition_groups(X, cutoff=0.5)
    
    # simulate y
    βtrue = zeros(p)
    βtrue[1:k] .= rand(-1:2:1, k) .* 0.25
    shuffle!(βtrue)
    correct_groups = groups[findall(!iszero, βtrue)] |> unique
    ϵ = randn(n)
    y = X * βtrue + ϵ;

    # group ME knockoffs
    t = @elapsed ko_filter = fit_lasso(y, X, method=:maxent, groups=groups, m=5)
    selected = ko_filter.selected[3]
    power = length(intersect(correct_groups, selected)) / length(correct_groups)
    fdr = length(setdiff(selected, correct_groups)) / max(1, length(selected))
    println("Sim $sim group-knockoff power = $power, FDR = $fdr, time=$t")
    push!(group_powers, power); push!(group_fdrs, fdr); push!(group_times, t)
    GC.gc();GC.gc();GC.gc();
end

println("\nME group knockoffs have average group power $(mean(group_powers))")
println("ME group knockoffs have average group FDR $(mean(group_fdrs))")
println("ME group knockoffs took average $(mean(group_times)) seconds");

Sim 1 group-knockoff power = 1.0, FDR = 0.1, time=6.71429675
Sim 2 group-knockoff power = 0.7777777777777778, FDR = 0.0, time=7.203738083
Sim 3 group-knockoff power = 0.8888888888888888, FDR = 0.1111111111111111, time=5.199876167
Sim 4 group-knockoff power = 0.8, FDR = 0.0, time=7.725970875
Sim 5 group-knockoff power = 0.7, FDR = 0.0, time=8.715202042
Sim 6 group-knockoff power = 0.5, FDR = 0.0, time=8.797519166
Sim 7 group-knockoff power = 1.0, FDR = 0.0, time=6.052631459
Sim 8 group-knockoff power = 0.4444444444444444, FDR = 0.0, time=8.221799459
Sim 9 group-knockoff power = 0.7, FDR = 0.0, time=9.489696541
Sim 10 group-knockoff power = 0.5555555555555556, FDR = 0.0, time=6.281113166

ME group knockoffs have average group power 0.7366666666666667
ME group knockoffs have average group FDR 0.021111111111111112
ME group knockoffs took average 7.4401843708 seconds


For comparison, lets try the same simulation but we generate regular (non-grouped) knockoffs

In [30]:
regular_powers, regular_fdrs, regular_times = Float64[], Float64[], Float64[]

Random.seed!(2022)
for sim in 1:10
    # simulate X
    Random.seed!(sim)
    n = 1000 # sample size
    p = 200  # number of covariates
    k = 10   # number of true predictors
    Σ = Matrix(SymmetricToeplitz(0.9.^(0:(p-1)))) # true covariance matrix
    μ = zeros(p)
    L = cholesky(Σ).L
    X = randn(n, p) * L
    zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X
    
    # simulate y
    βtrue = zeros(p)
    βtrue[1:k] .= rand(-1:2:1, k) .* 0.25
    shuffle!(βtrue)
    correct_snps = findall(!iszero, βtrue)
    ϵ = randn(n)
    y = X * βtrue + ϵ;

    # group ME knockoffs
    t = @elapsed ko_filter = fit_lasso(y, X, method=:maxent, m=5)
    selected = ko_filter.selected[3]
    power = length(intersect(correct_snps, selected)) / length(correct_snps)
    fdr = length(setdiff(selected, correct_snps)) / max(1, length(selected))
    println("Sim $sim nongroup-knockoff power = $power, FDR = $fdr, time=$t")
    push!(regular_powers, power); push!(regular_fdrs, fdr); push!(regular_times, t)
    GC.gc();GC.gc();GC.gc();
end

println("\nME (standard) knockoffs have average group power $(mean(regular_powers))")
println("ME (standard) knockoffs have average group FDR $(mean(regular_fdrs))")
println("ME (standard) knockoffs took average $(mean(regular_times)) seconds");

Sim 1 nongroup-knockoff power = 0.7, FDR = 0.2222222222222222, time=5.165706875
Sim 2 nongroup-knockoff power = 0.7, FDR = 0.0, time=5.707978708
Sim 3 nongroup-knockoff power = 0.2, FDR = 0.0, time=4.334730542
Sim 4 nongroup-knockoff power = 0.0, FDR = 0.0, time=6.279638458
Sim 5 nongroup-knockoff power = 0.2, FDR = 0.0, time=7.839875459
Sim 6 nongroup-knockoff power = 0.0, FDR = 0.0, time=7.261292667
Sim 7 nongroup-knockoff power = 0.0, FDR = 0.0, time=4.292064292
Sim 8 nongroup-knockoff power = 0.0, FDR = 0.0, time=7.985766
Sim 9 nongroup-knockoff power = 0.4, FDR = 0.0, time=8.667096167
Sim 10 nongroup-knockoff power = 0.5, FDR = 0.0, time=5.635861

ME (standard) knockoffs have average group power 0.26999999999999996
ME (standard) knockoffs have average group FDR 0.02222222222222222
ME (standard) knockoffs took average 6.3170010168 seconds


## Conclusion

+ When variables are highly correlated so that one cannot find exact discoveries, group knockoffs may be useful for improving power as it identifies whether a group of variables are non-null without having to pinpoint the exact discovery.
+ Group knockoffs control the group FDR to be below the target FDR level. 
+ Groups do not have to be contiguous
